In [1]:
#import os  
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D, Dense, BatchNormalization, Activation
from keras.layers import GlobalAveragePooling2D, MaxPooling2D, add
from keras.models import Model
from keras.layers import SeparableConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
weight_decay=1e-4

log_filepath  = './xception_he_wd_slim_thinner'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [4]:
# 36 convolutional layers are structured into 14 modules
def entryflow(x,params,top=False,last=False):
    # modules 2-4,13
    # params is (3,)
    # top = true means module 2, don't use relu
    if last:
        stride = (2,2)
    else:
        stride = (1,1)
    residual = Conv2D(params[0], (1, 1), strides=stride,padding='same',
                     kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    residual = BatchNormalization(momentum=0.9, epsilon=1e-5)(residual)
    if top:
        x = Activation('relu')(x)
    x = SeparableConv2D(params[1], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params[2], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = MaxPooling2D((3, 3), strides=stride,padding='same')(x)
    x = add([x, residual])
    return x
    
def middleflow(x,params):
    # modules 5-12, params is int
    residual = x
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = add([x, residual])
    return x

def exitflow(x,params):
    # modules 14 , params is (2,)
    x = SeparableConv2D(params[0], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params[1], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)   
    return x

In [5]:
def xception(img_input,shallow=False, classes=10):
    # modules 1
    x = Conv2D(32,(3, 3),strides=(2, 2),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3),strides=(1,1),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(364, (3, 3),strides=(1,1),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    # module 5-12
    for _ in range(4):
        x = middleflow(x,364)
    # module 13
    x = entryflow(x,(512,364,512),last=True)
    # module 14
    x = exitflow(x,(768,1024))
    # output
    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)
    return x

In [6]:
img_input=Input(shape=(32,32,3))
output = xception(img_input)
model=Model(img_input,output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('xception_he_wd_slim_thinner.h5')

Epoch 1/300
782/782 [==============================] - 71s 91ms/step - loss: 2.6713 - acc: 0.4755 - val_loss: 2.4793 - val_acc: 0.5548
Epoch 2/300
782/782 [==============================] - 67s 86ms/step - loss: 2.2308 - acc: 0.6312 - val_loss: 2.0943 - val_acc: 0.6833
Epoch 3/300
782/782 [==============================] - 68s 86ms/step - loss: 2.0160 - acc: 0.6995 - val_loss: 2.1725 - val_acc: 0.6661
Epoch 4/300
782/782 [==============================] - 68s 87ms/step - loss: 1.8745 - acc: 0.7445 - val_loss: 1.9197 - val_acc: 0.7328
Epoch 5/300
782/782 [==============================] - 68s 87ms/step - loss: 1.7728 - acc: 0.7696 - val_loss: 1.8014 - val_acc: 0.7696
Epoch 6/300
782/782 [==============================] - 68s 86ms/step - loss: 1.6828 - acc: 0.7933 - val_loss: 1.8484 - val_acc: 0.7406
Epoch 7/300
782/782 [==============================] - 67s 86ms/step - loss: 1.6032 - acc: 0.8109 - val_loss: 1.6740 - val_acc: 0.7964
Epoch 8/300
782/782 [==============================] - 

782/782 [==============================] - 67s 86ms/step - loss: 0.2415 - acc: 0.9996 - val_loss: 0.6240 - val_acc: 0.9202
Epoch 122/300
782/782 [==============================] - 67s 85ms/step - loss: 0.2407 - acc: 0.9997 - val_loss: 0.6234 - val_acc: 0.9215
Epoch 123/300
782/782 [==============================] - 67s 86ms/step - loss: 0.2401 - acc: 0.9995 - val_loss: 0.6129 - val_acc: 0.9225
Epoch 124/300
782/782 [==============================] - 67s 86ms/step - loss: 0.2392 - acc: 0.9997 - val_loss: 0.6223 - val_acc: 0.9217
Epoch 125/300
782/782 [==============================] - 67s 86ms/step - loss: 0.2382 - acc: 0.9998 - val_loss: 0.6110 - val_acc: 0.9227
Epoch 126/300
782/782 [==============================] - 67s 86ms/step - loss: 0.2376 - acc: 0.9996 - val_loss: 0.6163 - val_acc: 0.9229
Epoch 127/300
782/782 [==============================] - 67s 86ms/step - loss: 0.2366 - acc: 0.9998 - val_loss: 0.6158 - val_acc: 0.9219
Epoch 128/300
782/782 [==============================] 

Epoch 181/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1991 - acc: 0.9999 - val_loss: 0.6065 - val_acc: 0.9235
Epoch 182/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1986 - acc: 0.9999 - val_loss: 0.6106 - val_acc: 0.9233
Epoch 183/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1978 - acc: 1.0000 - val_loss: 0.6084 - val_acc: 0.9242
Epoch 184/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1972 - acc: 0.9999 - val_loss: 0.6122 - val_acc: 0.9229
Epoch 185/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1965 - acc: 0.9999 - val_loss: 0.6000 - val_acc: 0.9247
Epoch 186/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1959 - acc: 1.0000 - val_loss: 0.6053 - val_acc: 0.9242
Epoch 187/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1953 - acc: 0.9999 - val_loss: 0.5982 - val_acc: 0.9240
Epoch 188/300
782/782 [==================

Epoch 241/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1850 - acc: 0.9999 - val_loss: 0.5969 - val_acc: 0.9240
Epoch 242/300
782/782 [==============================] - 68s 87ms/step - loss: 0.1847 - acc: 1.0000 - val_loss: 0.5913 - val_acc: 0.9250
Epoch 243/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1847 - acc: 1.0000 - val_loss: 0.5935 - val_acc: 0.9235
Epoch 244/300
782/782 [==============================] - 68s 87ms/step - loss: 0.1846 - acc: 1.0000 - val_loss: 0.5975 - val_acc: 0.9244
Epoch 245/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1845 - acc: 1.0000 - val_loss: 0.5998 - val_acc: 0.9229
Epoch 246/300
782/782 [==============================] - 67s 86ms/step - loss: 0.1846 - acc: 1.0000 - val_loss: 0.6014 - val_acc: 0.9236
Epoch 247/300
782/782 [==============================] - 68s 87ms/step - loss: 0.1845 - acc: 1.0000 - val_loss: 0.5931 - val_acc: 0.9252
Epoch 248/300
782/782 [==================